In [46]:
import pandas as pd
import yfinance as yf
import json
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic

producer = KafkaProducer(
    bootstrap_servers="192.168.80.34:9092",
)
admin_client = KafkaAdminClient(bootstrap_servers="192.168.80.34:9092")
topic_name = "financials"

# Check if topic exists
topic_metadata = admin_client.list_topics()
if topic_name not in topic_metadata:
    # Create topic
    topic = NewTopic(name=topic_name, num_partitions=4, replication_factor=2)
    admin_client.create_topics(new_topics=[topic])


companies: pd.DataFrame = pd.read_csv("list_companies.csv")

for i in range(1, len(companies)):
    df = (
        yf.Ticker(companies["ticker"][i])
        .history(
            period="5d",
        )
        .iloc[-3:]
    )
    df["Ticker"] = companies["ticker"][i]
    df["CIK"] = companies["CIK"][i]
    df.reset_index(inplace=True)
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

    for i in range(3):
        producer.send("financials", df.iloc[i].to_json().encode("utf-8"))
